In [1]:
from model import LinUCB
from ipywidgets import interact, widgets, Output, Layout
from IPython.display import YouTubeVideo

from random import randrange
import pandas as pd
import numpy as np
from utils import load_data, preprocess_fast, preprocess_without_title, preprocess_with_title, search_duck_selenium, get_next_song_context

# Load and preprocess data
data = load_data("data/songs.csv")

songs_links = pd.read_csv("data/songs_links.csv")

In [7]:
features = preprocess_fast(data)

# Initialize your LinUCB model
num_actions = len(features)
num_features = features.shape[1]
alpha = 0.9  # You can adjust this value based on your needs
linucb_model = LinUCB(num_actions=num_actions, num_features=num_features, alpha=alpha)
chosen_action = randrange(num_actions)

# Store user ratings
user_ratings = []

# Define interactive GUI elements
song_title_display = Output(layout=Layout(height='150px'))  # Set the maximum height here
youtube_display = Output(layout=Layout(height='315px'))  # Set the maximum height for YouTube video

current_song = None
rating_slider = widgets.IntSlider(min=1, max=10, description='Rating:')
suggest_button = widgets.Button(description='Suggest Song')

# Display the GUI
display(song_title_display)
display(youtube_display)
display(rating_slider)
display(suggest_button)

def suggest_song(b):
    global chosen_action
    
    # Get the next song context
    title, context = get_next_song_context(data, features, chosen_action)

    with song_title_display:
        print(f"Suggested Song Title: {title}")

    # Get YouTube link
    youtube_link = songs_links[songs_links['Title'] == title]['link'].values[0]
    
    # Display YouTube video
    with youtube_display:
        display(YouTubeVideo(youtube_link))

    # Get user rating
    rating = rating_slider.value
    user_ratings.append((title, rating))
    
    with song_title_display:
        print(user_ratings[-1])

    # Choose the action (song) based on the model
    chosen_action = linucb_model.choose_action(context)

    # Update the model with the user feedback
    linucb_model.update_model(context=context, action=chosen_action, reward=rating)

# Assign the suggest_song function to the button click event
suggest_button.on_click(suggest_song)


Output(layout=Layout(height='150px'))

Output(layout=Layout(height='315px'))

IntSlider(value=1, description='Rating:', max=10, min=1)

Button(description='Suggest Song', style=ButtonStyle())